# **Importing from xl "ib97" to get "df_mf_ib97_nn_s"  & xl "ib97" to get "df_mf_ib97_nn_s"** 

# Set up

In [50]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")

/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


In [51]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
from collections.abc import Mapping
import re
#from sys_funcs import 

In [52]:
# set print rows  This worksheet sets maximum # of rows printed
pd.set_option('display.max_rows', 1000)  # Adjust the number of rows to display
# pd.reset_option('display.max_rows')  
print('print set to 1000 rows max' )

print set to 1000 rows max


In [53]:
print("NOTE: timestamp = Test Date / Time does not work  use computed time stamp")

NOTE: timestamp = Test Date / Time does not work  use computed time stamp


# Def functions called in data importing & refinment.

In [54]:
media_lst = [
    "timestamp",
    "dtv",
    "ib_id",
    "cls",
    "cmmnts"
]

In [55]:
# 3rd version def drop_duplicates_by_test_time(df, keep='first', log=True):
def drop_duplicates_by_test_time(df, keep='first', log=True):
    """
    Removes duplicate rows based on the 'Test Date / Time' column.
    Keeps only the first (or last) occurrence.
    """
    df = df.copy()

    # Identify duplicate timestamps (beyond the one we keep)
    dupes = (
        df.loc[df.duplicated(subset=['Test Date / Time'], keep=keep), 'Test Date / Time']
        .astype(str)
        .values
        .tolist()
    )

    # Drop duplicate rows
    df = df.drop_duplicates(subset=['Test Date / Time'], keep=keep)

    # Optional logging
    if log and dupes:
        print("Removed duplicate rows for timestamps:", dupes)

    return df


In [56]:
# def strip_numbers_from_columns(df):
import re

def strip_numbers_from_columns(df):
    """
    Removes leading/trailing numbers and any leftover separators
    so that cases like '1.0ID' become 'ID'.
    """
    df = df.copy()
    new_cols = {}

    for col in df.columns:
        cleaned = col

        # Remove leading numbers + separators
        cleaned = re.sub(r'^\d+[\s\-\_\.:]*', '', cleaned)

        # Remove trailing numbers + separators
        cleaned = re.sub(r'[\s\-\_\.:]*\d+$', '', cleaned)

        # Remove leftover leading/trailing punctuation (.,-_:) after number removal
        cleaned = re.sub(r'^[\.\-\_\:]+', '', cleaned)
        cleaned = re.sub(r'[\.\-\_\:]+$', '', cleaned)

        new_cols[col] = cleaned

    return df.rename(columns=new_cols)


In [57]:
# Use drop duplicate function If duplicates are found
def drop_duplicate_columns(df, keep='first', log=True):
    """
    Removes duplicate column names from a DataFrame, keeping only the first
    (or last) occurrence. Useful after column-cleaning steps that may cause
    collisions. good i'm moving this around because I want to go ahead and do the I'm talking too

    Parameters
    ----------
    df : pandas.DataFrame
        Input DataFrame.
    keep : {'first', 'last'}, default 'first'
        Which duplicate to keep.
    log : bool, default True
        Whether to print which columns were removed.

    Returns
    -------
    pandas.DataFrame
        DataFrame with duplicate columns removed.
    """
    df = df.copy()

    # Identify duplicates beyond the one we keep
    dupes = df.columns[df.columns.duplicated(keep=keep)].tolist()

    # Drop them
    df = df.loc[:, ~df.columns.duplicated(keep=keep)]

    # Optional logging
    if log and dupes:
        print("Removed duplicate columns:", dupes)

    return df


In [58]:
# def prepend_empty_columns(df, col_list):
def prepend_empty_columns(df, col_list):
    """
    Prepend empty columns (from col_list) to the front of df.
    Returns a new DataFrame.
    """
    import pandas as pd

    # Create empty columns with same row count
    empty_df = pd.DataFrame(
        {col: [None] * len(df) for col in col_list}
    )

    # Prepend them
    return pd.concat([empty_df, df], axis=1)


In [59]:
# revised def fill_ib_media_cols(df):
def fill_ib_media_cols(df):
    df = df.copy()

    # --- 1. Clean and parse timestamp --------------------------
    def fix_ts(x):
        if pd.isna(x):
            return np.nan
        # Convert float → int safely
        try:
            x_int = int(float(x))
        except:
            return np.nan
        # Zero‑pad to 14 digits (YYYYMMDDHHMMSS)
        s = str(x_int).zfill(14)
        return pd.to_datetime(s, format="%Y%m%d%H%M%S", errors="coerce")

    df['timestamp'] = df['Test Date / Time'].apply(fix_ts)

    # --- 2. dtv ------------------------------------------------
    origin = pd.Timestamp("1900-01-01")
    df['dtv'] = (df['timestamp'] - origin).dt.days

    # --- 3. ib_id ----------------------------------------------
    def classify_ib_id(ts):
        if pd.isna(ts):
            return np.nan
        hour = ts.hour
        return "mrn" if 3 <= hour <= 12 else "eve"

    df['ib_id'] = df['timestamp'].apply(classify_ib_id)

    # --- 4–5. cls, cmmnts --------------------------------------
    df['cls'] = np.nan
    df['cmmnts'] = np.nan

    return df


In [60]:
# Sort the rows by timestamp
def sort_by_timestamp(df):
    """
    Sorts an InBody dataframe by the 'timestamp' column
    in ascending chronological order.
    """
    df = df.copy()
    df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    return df


In [61]:
# A function to combine frames of ib97 with IB97 and sorting into one data frame. N
# this will be used on a column by column basis for a list of columns.
def combine_weight_frames(df_a, df_b, ts_col="timestamp", wt_a="2. wt", wt_b="4. wt"):
    """
    Combines two dataframes with different weight column names into a single
    dataframe with columns: timestamp, wt, sorted by timestamp.

    Parameters
    ----------
    df_a : pd.DataFrame
        First dataframe containing a timestamp column and a weight column.
    df_b : pd.DataFrame
        Second dataframe containing a timestamp column and a weight column.
    ts_col : str, optional
        Name of the timestamp column (default 'timestamp').
    wt_a : str, optional
        Weight column name in df_a (default '2. wt').
    wt_b : str, optional
        Weight column name in df_b (default '4. wt').

    Returns
    -------
    pd.DataFrame
        Combined dataframe with columns: timestamp, wt, sorted by timestamp.
    """

    # Normalize df_a
    df_a_norm = df_a[[ts_col, wt_a]].rename(columns={wt_a: "wt"})

    # Normalize df_b
    df_b_norm = df_b[[ts_col, wt_b]].rename(columns={wt_b: "wt"})

    # Stack them vertically
    df_combined = pd.concat([df_a_norm, df_b_norm], ignore_index=True)

    # Sort by timestamp
    df_combined = df_combined.sort_values(by=ts_col).reset_index(drop=True)

    return df_combined


In [112]:
# def drop_duplicate_source_files(df, keep="first"):
def drop_duplicate_source_files(df, keep="first"):
    """
    Remove rows where 'source_file' appears more than once.
    keep='first' keeps the first occurrence,
    keep='last' keeps the last occurrence.
    """
    if "source_file" not in df.columns:
        raise ValueError("Column 'source_file' not found in DataFrame")

    return df.drop_duplicates(subset=["source_file"], keep=keep).reset_index(drop=True)


In [62]:
# remove ROW duplicates on the basis of timestamp
def remove_ib_duplicates(df, subset_cols=None):
    """
    Removes duplicate InBody rows based on key identifying columns.
    Default behavior: remove duplicates based on ['ID', 'timestamp'].
    """
    df = df.copy()

    # Default duplicate definition
    if subset_cols is None:
        subset_cols = ['timestamp']
        # subset_cols = ['ID', 'timestamp']
    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=subset_cols, keep='first')

    # Reset index for cleanliness
    df = df.reset_index(drop=True)

    return df


In [63]:
# remove ROW duplicates on the basis of "Test Date / Time"
def remove_ib_duplicates_Test_Date_Time(df, subset_cols=None):
    """
    Removes duplicate InBody rows based on key identifying columns.
    Default behavior: remove duplicates based on ['ID', 'Test Date / Time'].
    """
    df = df.copy()

    # Default duplicate definition
    if subset_cols is None:
        subset_cols = ['Test Date / Time']
        # subset_cols = ['ID', 'Test Date / Time']
    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=subset_cols, keep='first')

    # Reset index for cleanliness
    df = df.reset_index(drop=True)

    return df


In [64]:
# veirfy if rows exixt in master_timestamps(df_master, df_new, ts_col="timestamp"):
def filter_new_rows_by_master_timestamps(df_master, df_new, ts_col="timestamp"):
    """
    Filters df_new so that only rows whose timestamps appear in df_master remain.

    Parameters
    ----------
    df_master : pd.DataFrame
        The master dataframe containing valid timestamps.
    df_new : pd.DataFrame
        The new dataframe to be filtered.
    ts_col : str, optional
        The name of the timestamp column (default is 'timestamp').

    Returns
    -------
    pd.DataFrame
        A filtered version of df_new containing only rows whose timestamps
        exist in df_master.
    """

    # Extract the set of valid timestamps from the master dataframe
    valid_timestamps = set(df_master[ts_col])

    # Filter df_new to keep only rows with timestamps in the master set
    df_filtered = df_new[df_new[ts_col].isin(valid_timestamps)].copy()

    return df_filtered


In [65]:
# def keep_only_new_timestamps(df_master, df_new, ts_col="timestamp")
def keep_only_new_timestamps(df_master, df_new, ts_col="timestamp"):
    """
    Returns only the rows in df_new whose timestamps do NOT exist in df_master.

    Parameters
    ----------
    df_master : pd.DataFrame
        The master dataframe containing timestamps already ingested.
    df_new : pd.DataFrame
        The new dataframe to be filtered.
    ts_col : str, optional
        The name of the timestamp column (default is 'timestamp').

    Returns
    -------
    pd.DataFrame
        A filtered version of df_new containing only rows with timestamps
        NOT present in df_master.
    """

    # Extract the set of timestamps already in the master
    existing_ts = set(df_master[ts_col])

    # Keep only rows whose timestamp is NOT in the master
    df_filtered = df_new[~df_new[ts_col].isin(existing_ts)].copy()

    return df_filtered


In [66]:
# def append_rows_with_master_schema(master_df, adder_df):
import pandas as pd
import numpy as np

def append_rows_with_master_schema(master_df, adder_df):
    """
    Appends rows from adder_df into master_df while enforcing the master_df schema.

    For each row in adder_df:
      - Columns that exist in adder_df are copied.
      - Columns missing from adder_df are filled with NaN.
      - All master_df columns are preserved in order.

    Parameters
    ----------
    master_df : pd.DataFrame
        The master dataframe with the full schema.
    adder_df : pd.DataFrame
        The dataframe containing rows to append (subset of master columns).

    Returns
    -------
    pd.DataFrame
        Updated master_df with new rows appended.
    """

    # Reindex adder_df to match master_df columns, filling missing columns with NaN
    adder_aligned = adder_df.reindex(columns=master_df.columns)

    # Append and return
    return pd.concat([master_df, adder_aligned], ignore_index=True)


In [67]:
# def filter_by_value(df, column, value):
def filter_by_value(df, column, value):
    """
    Returns a filtered DataFrame containing only rows where df[column] == value.

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe to filter.
    column : str
        The column name to filter on.
    value : any
        The value that the column must match.

    Returns
    -------
    pd.DataFrame
        A filtered dataframe containing only matching rows.
    """
    return df[df[column] == value].copy()


In [68]:
# def write_df_to_pickle(df, filename):
def write_df_to_pickle(df, filename):
    """
    Writes a DataFrame to a pickle file.

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe to save.
    filename : str
        The pickle filename, e.g. 'mydata.pkl'.
    """
    df.to_pickle(filename)

# usage 
# write_df_to_pickle(df, "df.pkl")


In [69]:
# def load_df_from_pickle(filename):

def load_df_from_pickle(filename):
    """
    Loads a DataFrame from a pickle file.

    Parameters
    ----------
    filename : str
        Path to the pickle file.

    Returns
    -------
    pd.DataFrame
    """
    return pd.read_pickle(filename)

    # usage 
    # df = load_df_from_pickle("df.pkl")



In [70]:
# def scale_mean_to_one(series):
def scale_mean_to_one(series):
    """Scale a Pandas Series so that its mean becomes 1."""
    mean_val = series.mean()
    return series / mean_val


In [71]:
# def plot_column(df, col_name):
import matplotlib.pyplot as plt

def plot_column(df, col_name):
    """
    Plot a single column from a dataframe.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the column.
    col_name : str
        The name of the column to plot.
    """
    plt.figure(figsize=(10, 4))
    plt.plot(df[col_name], marker='o', linestyle='-', linewidth=1)
    plt.title(f"{col_name} over index")
    plt.xlabel("Index")
    plt.ylabel(col_name)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [72]:
# Universal csv loader that can accept all csv encoders
# def universal_import(folder_path,
import pandas as pd
from pathlib import Path
import csv

def universal_import(
    folder_path,
    pattern="*.csv",
    expected_columns=None,
    df_name=None,
    verbose=True
):
    folder = Path(folder_path)
    if not folder.exists():
        raise FileNotFoundError(f"Folder not found: {folder_path}")

    dfs = []
    #files = list(folder.glob(pattern))
    #files = [f for f in folder.glob(pattern) if not f.name.endswith(".utf8.csv")]
    files = [
        f for f in folder.glob(pattern)
        if not f.name.lower().endswith(".utf8.csv")
           and not f.name.lower().endswith("_utf8.csv")
]


    for f in files:
        if verbose:
            print(f"\n📄 Processing: {f.name}")

        # --- 1. Detect encoding ---
        try:
            with open(f, "r", encoding="utf-8") as fh:
                sample = fh.read(4096)
            encoding_used = "utf-8"
        except Exception:
            encoding_used = "ISO-8859-1"

        # --- 2. Detect delimiter ---
        try:
            with open(f, "r", encoding=encoding_used) as fh:
                sample = fh.read(4096)
                dialect = csv.Sniffer().sniff(sample)
                delimiter = dialect.delimiter
        except Exception:
            delimiter = ","  # safe fallback

        # --- 3. Load CSV ---
        try:
            df = pd.read_csv(f, encoding=encoding_used, sep=delimiter)
        except Exception as e:
            if verbose:
                print(f"❌ Failed to load {f.name}: {e}")
            continue

        # --- 4. Schema validation (optional) ---
        if expected_columns and df.shape[1] != expected_columns:
            if verbose:
                print(f"⚠️ Skipped {f.name}: expected {expected_columns} cols, got {df.shape[1]}")
            continue

        # --- 5. Normalize to UTF-8 for future stability ---
        utf8_path = f.with_suffix(".utf8.csv")
        df.to_csv(utf8_path, index=False, encoding="utf-8")

        # --- 6. Add metadata ---
        #df["source_file"] = f.name
        #df["encoding_used"] = encoding_used
        #df["delimiter_used"] = delimiter
        # --- 6. Add metadata safely ---
        if "source_file" not in df.columns:
            df["source_file"] = f.name
        
        if "encoding_used" not in df.columns:
            df["encoding_used"] = encoding_used
        
        if "delimiter_used" not in df.columns:
            df["delimiter_used"] = delimiter


        dfs.append(df)

        if verbose:
            print(f"✅ Loaded {f.name} | encoding={encoding_used} | delimiter='{delimiter}'")
            print(f"💾 Normalized UTF-8 saved as: {utf8_path.name}")

    # --- 7. Combine all files ---
    if not dfs:
        if verbose:
            print("\n🚫 No valid files loaded.")
        return pd.DataFrame()

    combined = pd.concat(dfs, ignore_index=True)

    # --- 8. Save combined pickle ---
    label = df_name if df_name else "imported_dataframe"
    pickle_path = Path.cwd() / f"{label}.pkl"
    combined.to_pickle(pickle_path)

    if verbose:
        print(f"\n📦 Combined DataFrame: {len(combined)} rows from {len(dfs)} files.")
        print(f"💾 Saved combined pickle: {pickle_path}")

    return combined


In [91]:
# NOTE this works only for load_ib97_folder(folder_path)
import pandas as pd
from pathlib import Path

def load_ib97_folder(folder_path):
    folder = Path(folder_path)
    dfs = []

    for f in folder.glob("*.csv"):
        try:
            df = pd.read_csv(f, encoding="latin1", sep=",")
            df["source_file"] = f.name
            # print(df["Test Date / Time"])
            dfs.append(df)
            print(f"Loaded: {f.name}")
        except Exception as e:
            print(f"Failed to load {f.name}: {e}")

    if not dfs:
        print("No valid CSV files found.")
        return pd.DataFrame()

    ib97_raw = pd.concat(dfs, ignore_index=True)
    print(f"\nFinal DataFrame: {len(ib97_raw)} rows from {len(dfs)} files.")
    return ib97_raw


# Import new data from ip77 in the data folder of repo and results =  "df_mf_ib97_nn_s"

## Creating "df_mf_ib97_nn_s"

### This segment imports the data from the Excel file probo/data/ib97 to dataframe with numbers in col names

In [ ]:
folder_path = "/home/bhuns/JL_2/data/ib97"
df_ib97_raw = universal_import(folder_path, pattern="*", expected_columns=None, df_name=None, verbose=True)

In [3]:
# "mstr_ib970_tst"
import pandas as pd

df_ib97_raw = pd.read_excel("mstr_ib970_tst.xlsm")

# verify 
list(df_ib97_raw.columns)

['1. Name',
 '2. ID',
 '3. Height',
 '4. Date of Birth',
 '5. Gender',
 '6. Age',
 '7. Mobile Number',
 '8. Phone Number',
 '9. Zip Code',
 '10. Address',
 '11. E-mail',
 '12. Date of Registration',
 '13. Memo',
 '14. Test Date / Time',
 '15. Weight',
 '16. Lower Limit (Weight Normal Range)',
 '17. Upper Limit (Weight Normal Range)',
 '18. TBW (Total Body Water)',
 '19. Lower Limit (TBW Normal Range)',
 '20. Upper Limit (TBW Normal Range)',
 '21. ICW (Intracellular Water)',
 '22. Lower Limit (ICW Normal Range)',
 '23. Upper Limit (ICW Normal Range)',
 '24. ECW (Extracellular Water)',
 '25. Lower Limit (ECW Normal Range)',
 '26. Upper Limit (ECW Normal Range)',
 '27. Protein',
 '28. Lower Limit (Protein Normal Range)',
 '29. Upper Limit (Protein Normal Range)',
 '30. Minerals',
 '31. Lower Limit (Minerals Normal Range)',
 '32. Upper Limit (Minerals Normal Range)',
 '33. DLM (Dry Lean Mass)',
 '34. BFM (Body Fat Mass)',
 '35. Lower Limit (BFM Normal Range)',
 '36. Upper Limit (BFM Normal

In [ ]:
df_ib97_raw_flt = filter_by_value(df_ib97_raw, '2. ID', "091725-1")
df_ib97_raw_flt


In [ ]:
df_ib97_raw_flt = filter_by_value(df_ib97_raw, '2. ID', "091725-1")
df_ib97_raw_flt


In [122]:
df_ib97_unique = drop_duplicate_source_files(df_ib97_raw_flt)


In [123]:
df_ib97_raw_flt[['1. Name', '2. ID','source_file','13. Test Date / Time']]


,1. Name,2. ID,source_file,13. Test Date / Time


In [76]:
# verify list(df_ib97_raw.columns)

In [77]:
#verify df_ib97_raw

### This segment strips off the col names of numbers and produces "df_m_ib97_nn" and demonstrates slicing

In [78]:
df_ib97_nn = strip_numbers_from_columns(df_ib97_raw)
#verify print(list(df_ib97_nn.columns))

In [79]:
# verify slicing df_ib97_nn[['ID','Test Date / Time','ECW/TBW']]  
#,"SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

In [81]:
drop_duplicate_columns(df_ib97_nn, keep='first', log=True)

Removed duplicate columns: ['Systolic', 'Diastolic', 'Pulse', 'Mean Artery Pressure', 'Pulse Pressure', 'Rate Pressure Product', 'Age', 'Mobile Number', 'Phone Number', 'Zip Code', 'Address', 'E-mail', 'Date of Registration', 'Memo', 'Test Date / Time', 'Weight', 'Lower Limit (Weight Normal Range)', 'Upper Limit (Weight Normal Range)', 'TBW (Total Body Water)', 'Lower Limit (TBW Normal Range)', 'Upper Limit (TBW Normal Range)', 'ICW (Intracellular Water)', 'Lower Limit (ICW Normal Range)', 'Upper Limit (ICW Normal Range)', 'ECW (Extracellular Water)', 'Lower Limit (ECW Normal Range)', 'Upper Limit (ECW Normal Range)', 'Protein', 'Lower Limit (Protein Normal Range)', 'Upper Limit (Protein Normal Range)', 'Minerals', 'Lower Limit (Minerals Normal Range)', 'Upper Limit (Minerals Normal Range)', 'DLM (Dry Lean Mass)', 'BFM (Body Fat Mass)', 'Lower Limit (BFM Normal Range)', 'Upper Limit (BFM Normal Range)', 'FFM (Fat Free Mass)', 'SMM (Skeletal Muscle Mass)', 'Lower Limit (SMM Normal Range

,Name,ID,Height,Date of Birth,Age,Mobile Number,Phone Number,Zip Code,Address,E-mail,...,adjusted SMI,M/F,50KH,33,jb,bjh,ï»¿1. Name,SMI(ASM/HtÂ²)_T score,SMI(ASM/HtÂ²)_Z score,WC(Waist Circumference)Â
0,251201-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,251201-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
remove_ib_duplicates_Test_Date_Time(df_ib97_nn, subset_cols=None)

,Name,ID,Height,Date of Birth,Age,Mobile Number,Phone Number,Zip Code,Address,E-mail,...,HGS/WT,Unnamed,50KH,33,jb,bjh,ï»¿1. Name,SMI(ASM/HtÂ²)_T score,SMI(ASM/HtÂ²)_Z score,WC(Waist Circumference)Â
0,251201-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,251201-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
df_ib97_nn["Test Date / Time"]

,Test Date / Time,Test Date / Time,Test Date / Time
0,2.025121e+13,NaN,NaN
1,NaN,2.025112e+13,NaN
2,NaN,2.025102e+13,NaN
3,NaN,2.025093e+13,NaN
4,NaN,2.025103e+13,NaN
5,NaN,2.025112e+13,NaN
6,NaN,2.025102e+13,NaN
7,NaN,2.025110e+13,NaN
8,2.025121e+13,NaN,NaN
9,NaN,2.025112e+13,NaN


In [84]:
#
df_ib97_nn_s = df_ib97_nn.sort_values(by="Test Date / Time").reset_index(drop=True)

ValueError: The column label 'Test Date / Time' is not unique.

In [ ]:
# verify df_ib97_nn_s[['ID','Test Date / Time','ECW/TBW']]

### This segment adds **media_cols** and fills them from data in the results **"df_m_ib97_nn"**

In [ ]:
df_m_ib97_nn = prepend_empty_columns(df_ib97_nn, media_lst)
# verify 
df_m_ib97_nn

In [ ]:
# fill_ib_media_cols(df_mf_ib97_nn)
df_mf_ib97_nn = fill_ib_media_cols(df_m_ib97_nn)
# verify df_mf_ib97_nn

### This segment eliminates "COL" duplicates in **df_mf_ib97_nn**

In [ ]:
df_mf_ib97_nn = drop_duplicate_columns(df_mf_ib97_nn)
print ("df_mf_ib97_nn , No numbers, no duplicates OK")

### This segment sorts "df_mf_ib97_nn" to get "df_mf_ib97_nn_s"

In [ ]:
df_mf_ib97_nn_s = sort_by_timestamp(df_mf_ib97_nn )
# verify df_mf_ib97_nn_s

### This segment eliminates "ROW" duplicates based on **"test_time" in **"df_mf_ib97_nn"** note: "mf" means media added and filled]

In [110]:
df_mf_ib97_nn_s = df_mf_ib97_nn_s.drop_duplicates(subset="timestamp", keep="first")
print("No timestamp duplicates **df_mf_ib97_nn** detected")

NameError: name 'df_mf_ib97_nn_s' is not defined

In [ ]:
# verify list(df_mf_ib97_nn_s.columns)

# Set the "plt_lst" [the list ov cols to be plotted]
1. **"df_mf_ib97_nn_s"** is the source data 

In [ ]:
plt_lst = list["ECW/TBW", 'BMR (Basal Metabolic Rate)',"SMM (Skeletal Muscle Mass)",'VFA (Visceral Fat Area)']
# verify 
plt_lst

In [ ]:
#verify df_mf_ib97_nn_s

In [ ]:
# Selects the cols to be plotted 
plt_lst = ["ECW/TBW", 'BMR (Basal Metabolic Rate)',"SMM (Skeletal Muscle Mass)",'VFA (Visceral Fat Area)']
df_mf_ib97_nn_s[["timestamp",'ib_id']+plt_lst]

In [ ]:
# This filters out only the morning values of chosen data Col
df_mf_ib97_nn_s_mrn = filter_by_value(df_mf_ib97_nn_s, 'ib_id', "mrn")
# verify df_mf_ib97_nn_s_mrn

In [ ]:
write_df_to_pickle(df_mf_ib97_nn_s, "df_mf_ib97_nn_s_mrn.pkl")
print("df_mf_ib97_nn_s_mrn written to pickle")

# Normalize and subract 1 on "df_mf_ib97_nn_s[plt_lst]" col by col

In [ ]:
# df_mf_ib97_nn_s_n0 = df_mf_ib97_nn_s.copy() 
df_mf_ib97_nn_s_mrn_n0 = df_mf_ib97_nn_s_mrn.copy() 

In [ ]:
# plt_col = "ECW/TBW"

In [ ]:
# df_mf_ib97_nn_s_n0[plt_col]  =   scale_mean_to_one(df_mf_ib97_nn_s[plt_col])-1

In [ ]:
# plot_column(df_mf_ib97_nn_s_n0, plt_col)

In [ ]:
plt_lst = ['Weight',"ECW/TBW", 'BMR (Basal Metabolic Rate)',"SMM (Skeletal Muscle Mass)",'VFA (Visceral Fat Area)']

In [ ]:
for plt_col in plt_lst:
    print("Now plotting:", plt_col)
    df_mf_ib97_nn_s_mrn_n0[plt_col]  =   scale_mean_to_one(df_mf_ib97_nn_s_mrn[plt_col])-1
    plot_column(df_mf_ib97_nn_s_mrn_n0, plt_col)

In [ ]:
# plot_column(df_mf_ib97_nn_s_n0, "ECW/TBW")

# WORKING cell 
1. 
